In [ ]:
print("🔧 Installing dependencies...\n")

# Install core dependencies
!pip install -q torch torchvision
!pip install -q ultralytics  # YOLO
!pip install -q boxmot  # Tracking
!pip install -q opencv-python-headless
!pip install -q pandas scikit-learn

# Install new dependencies for embedding-based assignment
!pip install -q transformers  # SigLip model
!pip install -q umap-learn  # Dimensionality reduction
!pip install -q pillow  # Image processing

# Other utilities
!pip install -q tqdm matplotlib

# GTA-Link dependencies (for tracklet refinement)
!pip install -q loguru seaborn Cython

print("\n✅ All dependencies installed!")
print("\n⚠️ If you see any errors, restart runtime and run this cell again.")

In [ ]:
import os
import sys

# Replace with your GitHub repository URL
GITHUB_REPO = "https://github.com/HiteshG/Eaglevision.git"

print("📦 Cloning repository...\n")

# Remove if exists
if os.path.exists('Eaglevision'):
    !rm -rf Eaglevision

# Clone repository
!git clone {GITHUB_REPO}

# Change to repository directory
%cd Eaglevision

# Add to Python path
sys.path.insert(0, '/content/Eaglevision')

print("\n✅ Repository cloned successfully!")
print(f"\n📂 Working directory: {os.getcwd()}")
print("\n📋 Files in repository:")
!ls -la

# ============================================================================
# GTA-LINK INSTALLATION (Optional - for tracklet refinement)
# ============================================================================
USE_GTA_LINK = True  # Set to False to disable GTA-Link

if USE_GTA_LINK:
    print("\n" + "=" * 60)
    print("🔗 Installing GTA-Link for tracklet refinement...")
    print("=" * 60)
    
    # Clone GTA-Link repository
    !git clone https://github.com/sjc042/gta-link.git
    
    # Install torchreid from local folder
    %cd gta-link/reid
    !python setup.py develop --quiet
    %cd ../..
    
    # Add to path
    sys.path.insert(0, '/content/Eaglevision/gta-link')
    sys.path.insert(0, '/content/Eaglevision/gta-link/reid')
    
    # Verify model checkpoint exists
    model_path = "gta-link/reid_checkpoints/sports_model.pth.tar-60"
    if os.path.exists(model_path):
        print(f"\n✅ GTA-Link installed! Model found at: {model_path}")
    else:
        print(f"\n⚠️ GTA-Link installed but model not found at: {model_path}")
        print("   Refinement will be skipped if model is missing.")
    
    print("\n📋 GTA-Link files:")
    !ls -la gta-link/

In [ ]:
MODEL_PATH = "/content/detector_large_hd.pt"

In [ ]:
VIDEO_PATH = "/content/15sec_input_720p.mp4"

# Get video info
import cv2
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = frame_count / fps
cap.release()

print(f"\n📊 Video Information:")
print(f"   Resolution: {width}x{height}")
print(f"   FPS: {fps:.2f}")
print(f"   Frames: {frame_count}")
print(f"   Duration: {duration:.2f} seconds")


📊 Video Information:
   Resolution: 1280x720
   FPS: 25.00
   Frames: 375
   Duration: 15.00 seconds


### Setting config

In [ ]:
from config import MainConfig
import torch
from dataclasses import dataclass

# Team assignment method: "color" or "embedding"
TEAM_METHOD = "color"  # Change to "embedding" for SigLip-based assignment

# Processing FPS (lower = faster, higher = more accurate)
PROCESSING_FPS = 25

# Detection confidence threshold
CONFIDENCE = 0.35

# Memory decay (frames before forgetting team assignment)
MEMORY_DECAY = 150

# Visualization options
SHOW_BBOXES = True  # Show bounding boxes
SHOW_IDS = True      # Show player IDs

CLASS_NAMES =  {
    0: "Ball",
    1: "Goalkeeper",
    2: "Player",
    3: "Referee"
}

# ============================================================================
# CAMERA MOTION COMPENSATION & TRACKER SETTINGS
# ============================================================================

# CMC Method: "orb" (recommended), "ecc", "sof", "sparseOptFlow", or None (disabled)
CMC_METHOD = "orb"

# Track management (reduce ghost tracks)
TRACK_BUFFER = 30           # Frames to keep lost tracks (lower = fewer ghosts)
NEW_TRACK_THRESH = 0.6      # Confidence for new tracks (higher = fewer false tracks)
TRACK_HIGH_THRESH = 0.5     # High confidence association threshold
TRACK_LOW_THRESH = 0.1      # Low confidence association threshold
MATCH_THRESH = 0.8          # IoU matching threshold

# ============================================================================
# GTA-LINK SETTINGS (Tracklet Refinement)
# ============================================================================

@dataclass
class GTALinkConfig:
    """Configuration for GTA-Link post-processing."""
    model_path: str = "gta-link/reid_checkpoints/sports_model.pth.tar-60"
    # Split parameters (DBSCAN clustering)
    eps: float = 0.6           # DBSCAN epsilon (cosine distance threshold)
    min_samples: int = 10      # DBSCAN min points for core sample
    max_k: int = 3             # Max clusters after splitting
    min_len: int = 100         # Min tracklet length to attempt split
    # Merge parameters
    merge_dist_thres: float = 0.4  # Max cosine distance for merging
    spatial_factor: float = 1.0     # Scaling for spatial constraint ranges
    # Processing options
    use_split: bool = True     # Enable tracklet splitting
    use_connect: bool = True   # Enable tracklet merging
    batch_size: int = 64       # Batch size for feature extraction

gta_config = GTALinkConfig()

# ============================================================================
# ADVANCED SETTINGS (for embedding method)
# ============================================================================

if TEAM_METHOD == "embedding":
    EMBEDDING_BATCH_SIZE = 256  # Reduce if GPU out of memory
    EMBEDDING_STRIDE = 3        # Sample every N frames (higher = faster)
    SHRINK_SCALE = 0.7          # Focus on jersey area

# ============================================================================
# CREATE CONFIGURATION
# ============================================================================

config = MainConfig()

# Model settings
config.detector.model_path = MODEL_PATH
config.detector.confidence_threshold = CONFIDENCE

# Processing settings
config.fps = PROCESSING_FPS

# Camera Motion Compensation settings
config.tracker.cmc_method = CMC_METHOD
config.tracker.track_buffer = TRACK_BUFFER
config.tracker.new_track_thresh = NEW_TRACK_THRESH
config.tracker.track_high_thresh = TRACK_HIGH_THRESH
config.tracker.track_low_thresh = TRACK_LOW_THRESH
config.tracker.match_thresh = MATCH_THRESH

# Team assignment settings
config.team_assigner.team_method = TEAM_METHOD
config.team_assigner.memory_decay_frames = MEMORY_DECAY

# Embedding-specific settings
if TEAM_METHOD == "embedding":
    config.team_assigner.embedding_batch_size = EMBEDDING_BATCH_SIZE
    config.team_assigner.stride = EMBEDDING_STRIDE
    config.team_assigner.shrink_scale = SHRINK_SCALE

# Visualization settings
config.visualizer.show_bboxes = SHOW_BBOXES
config.visualizer.show_ids = SHOW_IDS

# Output directory
config.output_dir = "output"

# ============================================================================
# DISPLAY CONFIGURATION
# ============================================================================

print("⚙️ Configuration Summary")
print("=" * 60)
print(f"\n📹 Video: {os.path.basename(VIDEO_PATH)}")
print(f"🎯 Model: {os.path.basename(MODEL_PATH)}")
print(f"\n🔧 Processing Settings:")
print(f"   Team Method: {TEAM_METHOD.upper()}")
print(f"   FPS: {PROCESSING_FPS}")
print(f"   Confidence: {CONFIDENCE}")
print(f"   Memory Decay: {MEMORY_DECAY} frames ({MEMORY_DECAY/PROCESSING_FPS:.1f} seconds)")

print(f"\n📷 Camera Motion Compensation:")
print(f"   CMC Method: {CMC_METHOD if CMC_METHOD else 'DISABLED'}")
print(f"   Track Buffer: {TRACK_BUFFER} frames")
print(f"   New Track Threshold: {NEW_TRACK_THRESH}")
print(f"   Track High Threshold: {TRACK_HIGH_THRESH}")
print(f"   Match Threshold: {MATCH_THRESH}")

if TEAM_METHOD == "embedding":
    print(f"\n🧠 Embedding Settings:")
    print(f"   Batch Size: {EMBEDDING_BATCH_SIZE}")
    print(f"   Stride: {EMBEDDING_STRIDE}")
    print(f"   Shrink Scale: {SHRINK_SCALE}")

# GTA-Link settings
if USE_GTA_LINK:
    print(f"\n🔗 GTA-Link Settings (Tracklet Refinement):")
    print(f"   Model: {gta_config.model_path}")
    print(f"   Split: eps={gta_config.eps}, min_samples={gta_config.min_samples}, max_k={gta_config.max_k}")
    print(f"   Merge: threshold={gta_config.merge_dist_thres}, spatial_factor={gta_config.spatial_factor}")
else:
    print(f"\n🔗 GTA-Link: DISABLED")

print(f"\n🎨 Visualization:")
print(f"   Show Bounding Boxes: {SHOW_BBOXES}")
print(f"   Show Player IDs: {SHOW_IDS}")

# Check GPU availability
if torch.cuda.is_available():
    print(f"\n🚀 GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print(f"\n⚠️ No GPU available - using CPU (slower)")

print("\n" + "=" * 60)
print("\n✅ Configuration ready! Proceed to next cell to process video.")

## 🚀 Cell 6: Process Video

This will run the complete tracking pipeline:
1. **Video Reading** - Load and sample frames at target FPS
2. **Detection & Tracking** - YOLO detection + BoTSORT tracking
3. **GTA-Link Refinement** (Optional) - ReID-based tracklet split/merge
4. **Team Assignment** - Color or embedding-based clustering
5. **Data Processing** - Interpolation and ID merging
6. **Video Generation** - Create annotated output video

In [ ]:
cd /content/

/content


In [ ]:
from main import FootballTracker
from utils import read_video, create_output_directory, save_tracking_data, print_summary
from processor import DataProcessor
import time

# Import GTA-Link if enabled
if USE_GTA_LINK:
    from gta_link_processor import run_gta_link_refinement

print("🚀 Starting Football Tracker Pipeline")
print("=" * 60)
print(f"\n📹 Video: {os.path.basename(VIDEO_PATH)}")
print(f"🎯 Model: {os.path.basename(MODEL_PATH)}")
print(f"⚙️ Method: {TEAM_METHOD.upper()}")
print(f"🔗 GTA-Link: {'ENABLED' if USE_GTA_LINK else 'DISABLED'}")
print("\n" + "=" * 60 + "\n")

# Start timer
start_time = time.time()

try:
    # Initialize tracker
    tracker = FootballTracker(config)

    # Read video
    print("\n📖 Step 1/6: Reading video...")
    frames, fps = read_video(VIDEO_PATH, config.fps)
    output_dir = create_output_directory(VIDEO_PATH, config.output_dir)

    # Initialize processor with actual FPS
    tracker.processor = DataProcessor(config.processor, fps)

    # Detection and Tracking
    print("\n🔍 Step 2/6: Detecting and tracking objects...")
    detections_per_frame = tracker._detect_and_track(frames)

    # GTA-Link Refinement (Optional)
    if USE_GTA_LINK:
        print("\n🔗 Step 3/6: Running GTA-Link tracklet refinement...")
        detections_per_frame, _ = run_gta_link_refinement(
            detections_per_frame,
            frames,
            {},  # Empty team_mapping - will be assigned after refinement
            gta_config
        )
    else:
        print("\n⏭️ Step 3/6: Skipping GTA-Link refinement (disabled)")

    # Team Assignment
    print("\n👥 Step 4/6: Assigning teams...")
    team_mapping = tracker.team_assigner.assign_teams(frames, detections_per_frame)

    # Data Processing
    print("\n📊 Step 5/6: Processing tracking data...")
    df, team_mapping = tracker.processor.process(detections_per_frame, team_mapping)

    # Save Results and Create Video
    print("\n💾 Step 6/6: Saving results and creating video...")
    save_tracking_data(df, team_mapping, output_dir, fps)

    annotated_path = os.path.join(output_dir, "annotated.mp4")
    tracker.visualizer.create_annotated_video(
        frames, df, team_mapping, annotated_path, fps
    )

    print_summary(df, team_mapping, fps)

    # End timer
    elapsed_time = time.time() - start_time

    print("\n" + "=" * 60)
    print("✅ SUCCESS!")
    print("=" * 60)
    print(f"\n⏱️ Processing Time: {elapsed_time:.1f} seconds ({elapsed_time/60:.1f} minutes)")
    print(f"\n📁 Output Directory: {output_dir}")
    print(f"\n📊 Output Files:")

    # List output files
    import glob
    for file in sorted(glob.glob(f"{output_dir}/*")):
        size = os.path.getsize(file) / (1024 * 1024)  # MB
        print(f"   - {os.path.basename(file)} ({size:.2f} MB)")

    print("\n" + "=" * 60)
    print("\n✅ Ready to preview and download results!")

except Exception as e:
    print("\n" + "=" * 60)
    print("❌ ERROR OCCURRED")
    print("=" * 60)
    print(f"\n{str(e)}")
    print("\n📋 Troubleshooting:")
    print("   1. Check if video file is valid")
    print("   2. Try lower FPS (e.g., 12)")
    print("   3. If using 'embedding' method, try 'color' instead")
    print("   4. Reduce batch size if GPU out of memory")
    print("   5. Set USE_GTA_LINK = False to disable tracklet refinement")
    raise